In [1]:
import os
import csv

import cv2
import numpy as np
import pandas as pd
import sklearn

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
DATASET = "data_mine"

In [3]:
header = ["center", "left", "right", "steering", "throttle", "break", "speed"]
df = pd.read_csv(os.path.join(DATASET, "driving_log.csv"), header=None, names=header)
df.head()

,center,left,right,steering,throttle,break,speed
0,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000020
1,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000011
2,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000011
3,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000021
4,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000016


In [4]:
steering_labels = df.steering.values
center_images = df.center.values
left_images = df.left.values
right_images = df.right.values

In [5]:
samples = [left_images, center_images, right_images, steering_labels]

In [6]:
train_samples, validation_samples = train_test_split(df, test_size=0.2)

In [7]:
def read_img(name):
    img = name.split('/')[-1]
    return cv2.imread(os.path.join(DATASET,"IMG", img))

In [8]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            y_train = batch_samples.steering.values
            images = batch_samples.center.apply(read_img).values

            # define images for training
            X_train = images
            yield shuffle(X_train, y_train)

In [9]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)
ch, row, col = 3, 160, 320  # Trimmed image format

In [10]:
import tensorflow as tf

In [11]:
model = tf.keras.Sequential()

model.add(tf.layers.Conv2D(32, (3, 3), input_shape=(160, 320, 3), activation='relu'))
model.add(tf.layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(tf.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(tf.layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(tf.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(tf.layers.MaxPooling2D((2, 2), strides=(2,2)))

model.add(tf.layers.Flatten())
model.add(tf.layers.Dense(16, activation='relu'))

model.add(tf.layers.Dense(1))

In [12]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

In [13]:
def single_generator(samples):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        for i in range(0, num_samples):
            sample = samples[i:i+1]

            y_train = sample.steering.values[0]
            images = sample.center.apply(read_img).values[0]

            # define images for training
            X_train = images
            yield (X_train, y_train)

In [14]:
sample = train_samples[0:1]
sample.steering.values
sample.center.apply(read_img).values[0].shape

(160, 320, 3)

In [15]:
train_generator = single_generator(train_samples)
x,y=next(train_generator)
x.shape, y.shape
x.shape

(160, 320, 3)

In [91]:
dataset = tf.data.Dataset.from_generator(generator=lambda: train_generator,
                                         output_types=(tf.float32, tf.float32))

In [92]:
sess = tf.Session()
value = dataset.make_one_shot_iterator().get_next()
sess.run(value)

(array([[[197., 156., 117.],
         [205., 161., 124.],
         [208., 157., 125.],
         ...,
         [ 74., 131., 116.],
         [ 76., 117., 102.],
         [ 89., 124., 110.]],
 
        [[203., 164., 126.],
         [202., 160., 123.],
         [209., 159., 129.],
         ...,
         [ 77., 113.,  97.],
         [ 99., 117., 104.],
         [119., 131., 119.]],
 
        [[203., 165., 130.],
         [193., 153., 118.],
         [208., 164., 135.],
         ...,
         [145., 136., 122.],
         [170., 144., 132.],
         [180., 145., 135.]],
 
        ...,
 
        [[ 96., 112., 111.],
         [107., 123., 122.],
         [118., 134., 133.],
         ...,
         [111., 125., 124.],
         [124., 138., 137.],
         [135., 149., 148.]],
 
        [[130., 146., 145.],
         [139., 155., 154.],
         [145., 161., 160.],
         ...,
         [122., 136., 135.],
         [109., 123., 122.],
         [ 99., 113., 112.]],
 
        [[154., 170., 169.],
 

In [32]:
BATCH_SIZE = 32
EPOCHS = 2
steps_per_epoch = len(train_samples)//BATCH_SIZE
steps_per_epoch

111

#### dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.repeat()

In [95]:
model.fit(dataset, epochs=10, steps_per_epoch=steps_per_epoch)

ValueError: Cannot take the length of Shape with unknown rank.

In [28]:
filenames = train_samples.center.apply(lambda filename: os.path.join(DATASET,"IMG", filename.split("/")[-1])).values
labels = train_samples.steering.values

val_filenames = train_samples.center.apply(lambda filename: os.path.join(DATASET,"IMG", filename.split("/")[-1])).values
val_labels = train_samples.steering.values

In [18]:
def parse_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
#     image = tf.image.convert_image_dtype(image, tf.float32)

#     image = tf.image.resize_images(image, [160, 320])
    return image, label

In [33]:
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.shuffle(len(filenames))
dataset = dataset.map(parse_function, num_parallel_calls=4)
dataset = dataset.repeat(EPOCHS)
# dataset = dataset.map(train_preprocess, num_parallel_calls=4)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(1)

In [34]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_filenames, val_labels))
val_dataset = val_dataset.map(parse_function, num_parallel_calls=4)
val_dataset = val_dataset.repeat(EPOCHS)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(1)

In [25]:
sess = tf.Session()
value = dataset.make_one_shot_iterator().get_next()
x,y = sess.run(value)

In [26]:
y.shape

(32,)

In [35]:
model.fit(dataset,
          epochs=EPOCHS,
          steps_per_epoch=steps_per_epoch,
          validation_data=val_dataset,
          validation_steps=steps_per_epoch)

Epoch 1/2
111/111 [==============================] - 38s 342ms/step - loss: 0.0060 - mean_absolute_error: 0.0503 - val_loss: 0.0059 - val_mean_absolute_error: 0.0501
Epoch 2/2
111/111 [==============================] - 38s 342ms/step - loss: 0.0059 - mean_absolute_error: 0.0500 - val_loss: 0.0059 - val_mean_absolute_error: 0.0499


In [42]:
# Load an image
x = cv2.imread(val_filenames[0])
x[None, :, :, :].shape

(1, 160, 320, 3)

In [45]:
float(model.predict(x[None, :, :, :]))

-0.005988451186567545

In [37]:
tf.keras.models.save_model(
    model,
    "models/test1"
)

In [38]:
model_loaded = tf.keras.models.load_model(
    "models/test1",
)

In [46]:
float(model_loaded.predict(x[None, :, :, :]))

-0.005988451186567545